In [2]:
# Тест скорости чтения из таблицы лайков и отзывов о фильмах на Clickhouse

from clickhouse_driver import Client

client = Client(host='localhost')

In [23]:
import random, string
from datetime import datetime
from uuid import uuid4

movie_id_list = [uuid4() for _ in range(100)]
user_id_list = [uuid4() for _ in range(10000)]

def randomword(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(length))

def randomphrase(length):
    return ' '.join(randomword(random.randint(1, 10)) for _ in range(length))

def run_insert(n):
    record = {}
    for _ in range(0, n):
        record["date"] = datetime.now()
        record["user_id"] = str(random.choice(user_id_list))
        record["movie_id"] = str(random.choice(movie_id_list))
        record["score"] = random.randint(1, 10)
        record["mark"] = 0 if random.randint(1, 6)-5 < 0 else 1
        record["text"] = "None" if random.randint(1, 11)-10 < 0 else randomphrase(random.randint(5, 50))

        client.execute(
            'INSERT INTO simple.movieAnaliticsDb VALUES', [record]
        )

In [26]:
%timeit -n5 -r5 run_insert(100000)

In [ ]:
# Выберем несколько пользователей и фильмов для тестов
# user_id: '00020643-f40f-4fec-8ca0-3c30d81d0ceb'
# user_id: '000456f5-444e-4083-af32-dcd5e3fd61ba'
# movie_id: 'afb95da2-b945-46f1-b4ac-be82656650b5'
# movie_id: 'da43eb80-8f4f-4e09-a065-3f20f810dec6'
client.execute('SELECT * FROM simple.movieAnaliticsDb LIMIT 10')


In [66]:
# Найти фильмы понравившиеся одному пользователю
%timeit -n5 -r5 client.execute("SELECT movie_id, score FROM simple.movieAnaliticsDb WHERE user_id = '000456f5-444e-4083-af32-dcd5e3fd61ba' AND score >= 9")

4.18 ms ± 1.18 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [72]:
# Посчитать количество лайков и дизлайков у одного фильма
%timeit -n5 -r5 client.execute("SELECT count(score) FROM simple.movieAnaliticsDb WHERE movie_id = 'afb95da2-b945-46f1-b4ac-be82656650b5' AND score = 10")
%timeit -n5 -r5 client.execute("SELECT count(score) FROM simple.movieAnaliticsDb WHERE movie_id = 'afb95da2-b945-46f1-b4ac-be82656650b5' AND score = 1")

2.75 s ± 61.2 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
2.89 s ± 91.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [74]:
# Получение списка закладок для одного пользователя
%timeit -n5 -r5 client.execute("SELECT movie_id FROM simple.movieAnaliticsDb WHERE user_id = '00020643-f40f-4fec-8ca0-3c30d81d0ceb' AND mark = 1")

4.41 ms ± 465 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [83]:
# Подсчет средней пользовательской оценки фильма
def find_middle_score():
    a = client.execute("SELECT sum(score) FROM simple.movieAnaliticsDb WHERE movie_id = 'da43eb80-8f4f-4e09-a065-3f20f810dec6'")[0][0]
    b = client.execute("SELECT count(score) FROM simple.movieAnaliticsDb WHERE movie_id = 'da43eb80-8f4f-4e09-a065-3f20f810dec6'")[0][0]
    return a/b
%timeit -n5 -r5 find_middle_score()

4.19 s ± 168 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
